Disclaimer: os códigos utilizados neste arquivo para o scraping foram criados com a ajuda do ChatGPT.

O que fiz foi:

- Compartilhei o código fonte das páginas de ajuda da Noh apontando para onde encontrar os links dos artigos e informações como títulos e subtítulos de categoria para que o código do scraping fosse mais assertivo;
- Direcionei quais informações eu gostaria que estivessem presentes (e como) no arquivo json a ser gerado;
- Solicitei códigos "parciais" para testar em pequena escala antes de rodar o scraping com todo o site, o que me permitiu realizar melhorias incrementais antes de rodar a versão final;
    - Ex.: melhorias nos códigos para corrigir problemas de caracteres especiais nas strings;
- Solicitei maneiras de burlar que a Noh notasse meu scraping de alguma forma. Não sei o quão possível isso seria, ainda mais coletando uma quantidade tão pequena de dados (78 artigos da central de ajuda), mas eu quis reduzir riscos para não comprometer o projeto. Obs.: acho que não tem problema ter feito esse scraping..?? kkkk please don't arrest me!

Esse arquivo ipynb já é a versão final após os testes e iterações realizadas para o desenvolvimento do scraping.

In [2]:
import json
import unicodedata
import time
import random
import requests
from bs4 import BeautifulSoup

In [3]:
# url da página principal da FAQ do Noh
main_url = "https://www.noh.com.br/meajuda"

In [4]:
# removendo caracteres especiais e normalizando
def limpar_texto(texto):
    if texto:
        texto = unicodedata.normalize("NFKC", texto)  # normalizando as strings
        texto = texto.replace("\u200d", "").replace("\uFEFF", "").replace("\u2060", "")  # removendo caracteres invisíveis
    return texto.strip()

In [5]:
# requisição e encoding correto
response = requests.get(main_url)
response.encoding = "utf-8"  # encoding utf-8
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
# coletando as categorias da faq
categorias = []
categoria_tags = soup.select(".collection-item a.ajuda-block")  # pegando apenas os blocos de categorias

for tag in categoria_tags:
    href = tag["href"]
    titulo_tag = tag.find("h3", class_="ajuda-titulo-categoria")  # titulo da categoria
    subtitulo_tag = tag.find("div", class_="text-align-center")  # subtitulo da categoria

    titulo = limpar_texto(titulo_tag.get_text(strip=True)) if titulo_tag else ""
    subtitulo = limpar_texto(subtitulo_tag.get_text(strip=True)) if subtitulo_tag else ""

    categorias.append({
        "titulo_categoria": titulo,
        "subtitulo_categoria": subtitulo,
        "link_categoria": f"https://www.noh.com.br{href}"
    })

# exibindo os resultados
for categoria in categorias:
    print(f"{categoria['titulo_categoria']} - {categoria['subtitulo_categoria']}: {categoria['link_categoria']}")

Sobre Nohs - Saiba mais sobre a Noh e a nossa segurança: https://www.noh.com.br/ajuda-categorias/sobre-nohs
Conta - Tudo o que você precisa saber pra usar a Noh: https://www.noh.com.br/ajuda-categorias/conta
Cartão Noh - O cartão compartilhado para casais modernos do Brasil. Um passa, dois pagam!: https://www.noh.com.br/ajuda-categorias/cartao-noh
Boletos - Tire as suas dúvidas sobre os pagamentos no boleto: https://www.noh.com.br/ajuda-categorias/boletos
Pix - O que você precisa saber sobre o funcionamento do Pix na Noh: https://www.noh.com.br/ajuda-categorias/pix
Mensalidade - Tudo o que você precisa saber sobre a mensalidade da Noh: https://www.noh.com.br/ajuda-categorias/mensalidade


In [7]:
#função para extrair os links dos artigos dentro de cada categoria que obtivemos
def extrair_links_artigos(categoria):
    url = categoria["link_categoria"]
    response = requests.get(url)
    response.encoding = "utf-8"  
    soup = BeautifulSoup(response.text, "html.parser")

    artigos_links = []
    artigos_tags = soup.select(".ajuda-listalink") 

    for tag in artigos_tags:
        href = tag["href"]
        if href.startswith("/ajuda/"):  # todos os artigos "começam" com essa substring na url
            artigos_links.append(f"https://www.noh.com.br{href}")

    return artigos_links

In [8]:
# armazenando os artigos em um dicionário
todos_artigos = {}

# iterando cada categoria para pegar os artigos da central de ajuda
for categoria in categorias:
    print(f"\nExtraindo artigos da categoria: {categoria['titulo_categoria']}")
    artigos = extrair_links_artigos(categoria)

    todos_artigos[categoria["titulo_categoria"]] = {
        "subtitulo_categoria": categoria["subtitulo_categoria"],
        "link_categoria": categoria["link_categoria"],
        "artigos": artigos
    }

    # exibindo a quantidade de artigos encontrados por categoria
    print(f"{len(artigos)} artigos encontrados")

    # delay aleatório para evitar bloqueios pela Noh
    time.sleep(random.uniform(2, 4))

# exibindo os resultados
print("\nFinalizado, lista de artigos por categoria:")
for categoria, dados in todos_artigos.items():
    print(f"\n{categoria} ({len(dados['artigos'])} artigos)")
    for artigo in dados["artigos"][:5]:  # mostrando apenas os 5 primeiros
        print(f"  - {artigo}")


Extraindo artigos da categoria: Sobre Nohs
7 artigos encontrados

Extraindo artigos da categoria: Conta
35 artigos encontrados

Extraindo artigos da categoria: Cartão Noh
10 artigos encontrados

Extraindo artigos da categoria: Boletos
5 artigos encontrados

Extraindo artigos da categoria: Pix
14 artigos encontrados

Extraindo artigos da categoria: Mensalidade
7 artigos encontrados

Finalizado, lista de artigos por categoria:

Sobre Nohs (7 artigos)
  - https://www.noh.com.br/ajuda/o-que-e-a-noh
  - https://www.noh.com.br/ajuda/os-meus-dados-e-o-meu-dinheiro-estao-seguros
  - https://www.noh.com.br/ajuda/quanto-custa
  - https://www.noh.com.br/ajuda/seguranca-saiba-como-se-proteger
  - https://www.noh.com.br/ajuda/onde-encontro-meu-informe-de-rendimentos

Conta (35 artigos)
  - https://www.noh.com.br/ajuda/como-faco-pra-criar-conta
  - https://www.noh.com.br/ajuda/quem-pode-criar-uma-conta-na-noh
  - https://www.noh.com.br/ajuda/sou-estrangeiro-posso-ter-uma-conta
  - https://www.noh.c

In [9]:
# extraindo o conteúdo de um artigo
def extrair_conteudo_artigo(url):
    response = requests.get(url)
    response.encoding = 'utf-8'  
    soup = BeautifulSoup(response.text, 'html.parser')

    # extraindo título e limpando texto
    titulo = soup.find('h1').get_text(strip=True) if soup.find('h1') else None
    titulo = limpar_texto(titulo)

    # extraindo conteúdo e limpando texto
    conteudo_tag = soup.find(class_='ajuda-artigo-texto w-richtext')
    conteudo = conteudo_tag.get_text("\n", strip=True) if conteudo_tag else None
    conteudo = limpar_texto(conteudo)

    # extraindo links da parte "artigos relacionados" contida em cada artigo
    artigos_relacionados = []
    related_articles = soup.select(".ajuda-listalink a")
    for link in related_articles:
        href = link.get("href")
        if href and href.startswith("/ajuda/"):
            artigos_relacionados.append(f"https://www.noh.com.br{href}")

    return {
        "url": url,
        "titulo": titulo,
        "conteudo": conteudo,
        "artigos_relacionados": artigos_relacionados
    }

In [10]:
# lista com todos os artigos disponíveis em todas as categorias
todos_links_artigos = []

for categoria, info_categoria in todos_artigos.items():
    for artigo_url in info_categoria["artigos"]:
        todos_links_artigos.append({
            "categoria": categoria,
            "subtitulo_categoria": info_categoria["subtitulo_categoria"],
            "link_categoria": info_categoria["link_categoria"],
            "url": artigo_url
        })

# armazenando os dados extraídos em uma lista
dados_artigos = []

# loop iterando na lista de links de artigos
for i, artigo in enumerate(todos_links_artigos, 1):
    print(f"({i}/{len(todos_links_artigos)}) Extraindo: {artigo['url']}")
    
    # extraindo conteúdo do artigo
    dados = extrair_conteudo_artigo(artigo["url"])

    if dados:
        # salvando as informações da categoria do artigo (assunto macro)
        dados["categoria"] = artigo["categoria"]
        dados["subtitulo_categoria"] = artigo["subtitulo_categoria"]
        dados["link_categoria"] = artigo["link_categoria"]

        # append na lista
        dados_artigos.append(dados)

    # delayzinho aleatório entre requisições para evitar bloqueios
    time.sleep(random.uniform(2, 4))

# salvando JSON com os dados extraídos
with open("noh_faq_completo_vf.json", "w", encoding="utf-8") as f:
    json.dump(dados_artigos, f, ensure_ascii=False, indent=4)

print("\nFinalizado, dados salvos em noh_faq_completo_vf.json")

(1/78) Extraindo: https://www.noh.com.br/ajuda/o-que-e-a-noh
(2/78) Extraindo: https://www.noh.com.br/ajuda/os-meus-dados-e-o-meu-dinheiro-estao-seguros
(3/78) Extraindo: https://www.noh.com.br/ajuda/quanto-custa
(4/78) Extraindo: https://www.noh.com.br/ajuda/seguranca-saiba-como-se-proteger
(5/78) Extraindo: https://www.noh.com.br/ajuda/onde-encontro-meu-informe-de-rendimentos
(6/78) Extraindo: https://www.noh.com.br/ajuda/como-e-declarado-o-imposto-de-renda-da-conta-conjunta
(7/78) Extraindo: https://www.noh.com.br/ajuda/como-funciona-o-programa-de-indicacao-da-noh
(8/78) Extraindo: https://www.noh.com.br/ajuda/como-faco-pra-criar-conta
(9/78) Extraindo: https://www.noh.com.br/ajuda/quem-pode-criar-uma-conta-na-noh
(10/78) Extraindo: https://www.noh.com.br/ajuda/sou-estrangeiro-posso-ter-uma-conta
(11/78) Extraindo: https://www.noh.com.br/ajuda/quais-documentos-preciso-enviar
(12/78) Extraindo: https://www.noh.com.br/ajuda/por-que-preciso-enviar-meus-documentos
(13/78) Extraindo: htt